[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

This dashboard displays data showing new coronavirus (Covid-19) cases in comparison to new hospital admissions and ventilator bed occupancy across the United Kingdom, from 30/01/2020 to 23/11/22.

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open('vent.json', 'rt') as INFILE:
    jsondata=json.load(INFILE)

In [4]:


def wrangle_data(data):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    datalist=data['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    def parse_date(datestring):   
         return pd.to_datetime(datestring, format="%Y-%m-%d")
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    ventdf=pd.DataFrame(index=index, columns=['cases', 'admissions', 'ventilationbeds'])
    for entry in datalist:
        date=parse_date(entry['date'])
        for column in ['cases', 'admissions', 'ventilationbeds']:
            if pd.isna(ventdf.loc[date, column]):
                value= float(entry[column]) if entry[column]!=None else 0.0 
                ventdf.loc[date, column]=value
            
    ventdf.fillna(0.0, inplace=True)

    
    
    return ventdf

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
ventdf=wrangle_data(jsondata) # df is the dataframe for plotting

## Download current data



Clicking on the button below refreshes the current Public Health England Data and refreshes the graph. 

In [5]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
    'areaType=nation',
    'areaName=England'
    ]
    structure = {
    "date": "date",
    "cases": "newCasesBySpecimenDate",
    "admissions": "newAdmissions",
    "ventilationbeds": "covidOccupiedMVBeds"
    }
    api = Cov19API(filters=filters, structure=structure)
    
    vent=api.get_json()
    return vent # return data read from the API



In [6]:

def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global ventdf
    ventdf=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Refresh data', 
    disabled=False,
    button_style='info', 
    tooltip="Click to download Public Health England Data",
    icon='download'
)

#  register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of the function is inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='info', description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click t…

## Graphs and Analysis


The graph below shows new coronavirus cases and new hospital admissions in comparison ventilated bed occupancy. The graph can be viewed in both linear and log format.

In [7]:
series=wdg.SelectMultiple(
    options=['cases', 'admissions', 'ventilationbeds'],
    value=['cases', 'admissions', 'ventilationbeds'],
    rows=3,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls=wdg.HBox([series, scale])

def vent_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        ventdf[list(gcols)].plot(logy=logscale)
        plt.show() # graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")


def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=scale.value
    if current==scale.options[0]:
        other=scale.options[1]
    else:
        other=scale.options[0]
    scale.value=other # forces the redraw
    scale.value=current # now we can change it back
    
graph=wdg.interactive_output(vent_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)

Output()

**Author and Copyright Notice** *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*  
**By Tyrique Charles**